In [ ]:
!pip install fastreid
!pip install mtcnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os 
import time, datetime, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as patches
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
sys.path.append('/content/drive/MyDrive/PyTorch-YOLOv3-master')
sys.path.append('/content/drive/MyDrive/deep_sort_pytorch-master')

In [ ]:
from pytorchyolo.models import*
from pytorchyolo.utils import*
from pytorchyolo.detect import*
config_path = '/content/drive/MyDrive/PyTorch-YOLOv3-master/config/yolov3.cfg'
weight_path = '/content/drive/MyDrive/3.weights'
class_path='/content/drive/MyDrive/PyTorch-YOLOv3-master/data/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

In [ ]:
#load model and weights
model = Darknet(config_path)
model.load_darknet_weights(weight_path)
model.cuda()
model.eval()
classes=utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
#feature extraction
sys.path.append('/content/drive/MyDrive')
from feature_extraction import extract_features

In [ ]:
from pytorchyolo.detect import*
from deep_sort.sort.detection import*
from deep_sort.sort.tracker import Tracker
from deep_sort.sort.kalman_filter import KalmanFilter
from deep_sort.sort.nn_matching import NearestNeighborDistanceMetric
from deep_sort import DeepSort
import cv2
from google.colab.patches import cv2_imshow
from mtcnn import MTCNN

# Create an instance of the MTCNN model
face_detector = MTCNN()

# Create VideoWriter object to save the processed frames as a video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 2.0, (640, 480))

# metric
metric = NearestNeighborDistanceMetric("cosine", 0.2, 100)
tracker = Tracker(metric, max_iou_distance=0.7, max_age=70, n_init=3)

# start streaming video from webcam
video_stream()

# label for video
label_html = 'Capturing...'

# initialize bounding box to empty
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    input_img = cv2.resize(img, (416, 416))

    #convert image to rgb for face detection
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces in the image
    faces = face_detector.detect_faces(image_rgb)
    
    # yolo detection
    detection = []
    detections = detect_image(model, img, img_size=416, conf_thres=0.5, nms_thres=0.5)
    for x1, y1, x2, y2, conf, cls_pred in detections:
      if classes[int(cls_pred)] == 'person':
        # extract features from an using roi and cnn model
        image_pil = Image.fromarray(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB))
        features = extract_features(image_pil, (x1, y1, x2, y2))

        # class and confidence score
        print(f"\t+ Label: {classes[int(cls_pred)]} | Confidence: {conf.item():0.4f}")
        box_w = x2 - x1
        box_h = y2 - y1

        # create a detection object and add it to the detection list
        detect_data = Detection([x1, y1, box_w, box_h], conf, features)
        detection.append(detect_data)
        
    # update the tracker with the detections
    tracker.predict()
    tracker.update(detection)

    # loop through the tracks and draw bounding boxes and labels on the frame
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        bbox = track.to_tlwh()
        track_id = track.track_id

        # draw bounding box
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])),
                      (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])),
                      (0, 255, 0), 2)
        # put text with track ID and class label
        cv2.putText(img, f"ID:{track_id}/Conf_Score:{conf:.2f}/Label:{classes[int(cls_pred)]}",
                    (int(bbox[0]), int(bbox[1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
      # Draw bounding boxes around the detected faces
        for face in faces:
            x, y, width, height = face['box']
            cv2.rectangle(img, (x, y), (x+width, y+height), (0, 255, 0), 2)
            cv2.putText(img, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    #saving results
    out.write(img)
cv2.destroyAllWindows()
 
